# 🏦 Credit Risk Scorecard — Notebook 2: Probability of Default (PD) Model

**Project:** Credit Risk Scorecard + IFRS 9 Expected Credit Loss (ECL) Engine  
**Input:** `cs-training-clean.csv` (from Notebook 1)  
**Objective:** Build a Logistic Regression model to estimate the probability of default (PD)

**Author:** *Your Name*  
**Date:** *2026*

---

### 📋 Notebook Contents
1. Load cleaned data
2. Feature engineering & preprocessing
3. Train / test split
4. Logistic Regression model
5. Model evaluation: AUC-ROC, KS Statistic, Confusion Matrix
6. Calibration curve
7. Feature importance
8. Save model & predictions

## Step 0: Upload Cleaned Data (Google Colab Only)

In [ ]:
# Uncomment if running in Google Colab
# from google.colab import files
# uploaded = files.upload()  # Upload cs-training-clean.csv

## Step 1: Import Libraries & Load Data

In [ ]:
# ── Core libraries ───────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# ── Scikit-learn ─────────────────────────────────────────────
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, roc_curve,
    confusion_matrix, classification_report,
    ConfusionMatrixDisplay
)
from sklearn.calibration import calibration_curve

# ── Save model ───────────────────────────────────────────────
import joblib

# Plot styling
plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['font.size'] = 12
sns.set_style('whitegrid')

print('All libraries imported successfully.')

In [ ]:
# ── Load cleaned data from Notebook 1 ───────────────────────
df = pd.read_csv('cs-training-clean.csv')

print(f'Data loaded: {df.shape[0]:,} rows x {df.shape[1]} columns')
print(f'Default rate: {df["default"].mean()*100:.2f}%')
df.head()

## Step 2: Feature Engineering

In [ ]:
# ── Create additional features ───────────────────────────────

# Total delinquency count (combined late payment signal)
df['total_late'] = df['late_30_59'] + df['late_60_89'] + df['late_90']

# Income per dependent (financial stress indicator)
# Add 1 to avoid division by zero
df['income_per_dependent'] = df['monthly_income'] / (df['dependents'] + 1)

# Log-transform skewed features to reduce the effect of extreme values
df['log_income'] = np.log1p(df['monthly_income'])   # log1p = log(1 + x), handles 0 safely
df['log_debt_ratio'] = np.log1p(df['debt_ratio'])

print('New features created:')
print('  · total_late          — sum of all late payment counts')
print('  · income_per_dependent — monthly income divided by (dependents + 1)')
print('  · log_income           — log-transformed monthly income')
print('  · log_debt_ratio       — log-transformed debt ratio')
print()
print(f'Total features: {df.shape[1] - 1}  (excluding target)')

## Step 3: Prepare Features & Split Data

In [ ]:
# ── Define features and target ───────────────────────────────
FEATURES = [
    'revolving_util',
    'age',
    'late_30_59',
    'late_60_89',
    'late_90',
    'open_credit_lines',
    'real_estate_loans',
    'dependents',
    'total_late',           # engineered
    'income_per_dependent', # engineered
    'log_income',           # engineered
    'log_debt_ratio',       # engineered
]

TARGET = 'default'

X = df[FEATURES]
y = df[TARGET]

# ── Train / test split (80% train, 20% test) ─────────────────
# random_state=42 ensures reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f'Training set : {X_train.shape[0]:,} rows  ({y_train.mean()*100:.2f}% default rate)')
print(f'Test set     : {X_test.shape[0]:,} rows  ({y_test.mean()*100:.2f}% default rate)')
print()
print('stratify=y ensures the default rate is balanced across both splits.')

In [ ]:
# ── Standardise features ─────────────────────────────────────
# Logistic Regression performs better when all features are on the same scale
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)   # fit on training data only
X_test_scaled  = scaler.transform(X_test)         # apply same scale to test data

print('Feature scaling complete (StandardScaler).')
print('Rule: fit on TRAIN only, transform both TRAIN and TEST.')
print('This prevents data leakage from the test set.')

## Step 4: Train Logistic Regression Model

In [ ]:
# ── Train model ──────────────────────────────────────────────
# class_weight='balanced' adjusts for the imbalanced dataset (93% non-default)
model = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)

model.fit(X_train_scaled, y_train)

print('Logistic Regression model trained successfully.')
print()
print('Key parameters:')
print('  · class_weight="balanced" — compensates for class imbalance')
print('  · max_iter=1000           — allows enough iterations to converge')
print('  · random_state=42         — ensures reproducible results')

## Step 5: Model Evaluation

In [ ]:
# ── Generate predictions ─────────────────────────────────────
# predict_proba gives probability scores (we use column 1 = P(default))
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
y_pred_class = model.predict(X_test_scaled)

# ── AUC-ROC ──────────────────────────────────────────────────
auc = roc_auc_score(y_test, y_pred_proba)

# ── KS Statistic ─────────────────────────────────────────────
# KS = max separation between cumulative default and non-default distributions
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
ks_stat = max(tpr - fpr)

print('=== Model Performance on Test Set ===')
print(f'  AUC-ROC      : {auc:.4f}   (target: > 0.75 = good, > 0.80 = very good)')
print(f'  KS Statistic : {ks_stat:.4f}   (target: > 0.30 = acceptable for credit scoring)')
print()
print('Classification Report:')
print(classification_report(y_test, y_pred_class, target_names=['Non-Default', 'Default']))

In [ ]:
# ── ROC Curve ────────────────────────────────────────────────
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='#3498db', linewidth=2.5,
         label=f'Logistic Regression (AUC = {auc:.4f})')
plt.plot([0, 1], [0, 1], color='grey', linestyle='--', linewidth=1, label='Random Classifier')

# Mark KS point
ks_idx = np.argmax(tpr - fpr)
plt.scatter(fpr[ks_idx], tpr[ks_idx], color='#e74c3c', zorder=5, s=100,
            label=f'KS = {ks_stat:.4f}  (threshold = {thresholds[ks_idx]:.3f})')

plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR / Recall)')
plt.title('ROC Curve — Logistic Regression PD Model', fontsize=14, fontweight='bold')
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig('plot_06_roc_curve.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: plot_06_roc_curve.png')

In [ ]:
# ── Confusion Matrix ─────────────────────────────────────────
cm = confusion_matrix(y_test, y_pred_class)

fig, ax = plt.subplots(figsize=(7, 5))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                               display_labels=['Non-Default', 'Default'])
disp.plot(ax=ax, colorbar=False, cmap='Blues')
ax.set_title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('plot_07_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

tn, fp, fn, tp = cm.ravel()
print(f'True Negatives  (correctly predicted Non-Default): {tn:,}')
print(f'False Positives (Non-Default predicted as Default): {fp:,}')
print(f'False Negatives (Default predicted as Non-Default): {fn:,}  ← costly in credit risk!')
print(f'True Positives  (correctly predicted Default)     : {tp:,}')
print('Saved: plot_07_confusion_matrix.png')

## Step 6: Calibration Curve

A calibration curve checks whether the model's predicted probabilities are reliable.  
If the model says P(default) = 20%, approximately 20% of those borrowers should actually default.  
This is critical for IFRS 9 ECL calculations — we need well-calibrated PD estimates.

In [ ]:
# ── Calibration curve ────────────────────────────────────────
prob_true, prob_pred = calibration_curve(y_test, y_pred_proba, n_bins=10)

plt.figure(figsize=(8, 6))
plt.plot(prob_pred, prob_true, 's-', color='#3498db', linewidth=2,
         markersize=8, label='Logistic Regression')
plt.plot([0, 1], [0, 1], '--', color='grey', linewidth=1, label='Perfect Calibration')

plt.xlabel('Mean Predicted Probability')
plt.ylabel('Fraction of Positives (Actual Default Rate)')
plt.title('Calibration Curve', fontsize=14, fontweight='bold')
plt.legend()
plt.tight_layout()
plt.savefig('plot_08_calibration_curve.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: plot_08_calibration_curve.png')
print()
print('Interpretation: Points close to the diagonal = well-calibrated model.')
print('Logistic Regression is generally well-calibrated by design.')

## Step 7: Feature Importance (Model Coefficients)

In [ ]:
# ── Logistic Regression coefficients ─────────────────────────
# Coefficient magnitude = feature importance (after standardisation)
coef_df = pd.DataFrame({
    'feature': FEATURES,
    'coefficient': model.coef_[0]
}).sort_values('coefficient', key=abs, ascending=True)

plt.figure(figsize=(9, 6))
colors = ['#e74c3c' if c > 0 else '#2ecc71' for c in coef_df['coefficient']]
plt.barh(coef_df['feature'], coef_df['coefficient'], color=colors, edgecolor='white')
plt.axvline(x=0, color='black', linewidth=0.8)
plt.xlabel('Coefficient Value')
plt.title('Feature Importance (Logistic Regression Coefficients)\nRed = increases default risk  |  Green = decreases default risk',
          fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('plot_09_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print('=== Feature Coefficients (sorted by absolute magnitude) ===')
coef_sorted = coef_df.sort_values('coefficient', key=abs, ascending=False)
for _, row in coef_sorted.iterrows():
    direction = '↑ risk' if row['coefficient'] > 0 else '↓ risk'
    print(f"  {direction}  {row['feature']:25s}  {row['coefficient']:+.4f}")
print('Saved: plot_09_feature_importance.png')

## Step 8: Score Distribution (PD Histogram)

In [ ]:
# ── Distribution of predicted PD by actual class ─────────────
plt.figure(figsize=(10, 5))

for label, color, name in [(0, '#2ecc71', 'Non-Default'), (1, '#e74c3c', 'Default')]:
    mask = y_test == label
    plt.hist(y_pred_proba[mask], bins=50, alpha=0.6, color=color,
             label=f'{name} (n={mask.sum():,})', density=True)

plt.xlabel('Predicted Probability of Default (PD)')
plt.ylabel('Density')
plt.title('PD Score Distribution by Actual Default Status', fontsize=14, fontweight='bold')
plt.legend()
plt.tight_layout()
plt.savefig('plot_10_score_distribution.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved: plot_10_score_distribution.png')
print()
print('Good separation between green and red = model discriminates well between defaults and non-defaults.')

## Step 9: Save Model & Predictions

In [ ]:
# ── Save trained model and scaler ────────────────────────────
joblib.dump(model, 'pd_model_logistic.pkl')
joblib.dump(scaler, 'pd_scaler.pkl')
print('Model saved  : pd_model_logistic.pkl')
print('Scaler saved : pd_scaler.pkl')

# ── Save test set predictions ─────────────────────────────────
results = X_test.copy()
results['actual_default'] = y_test.values
results['predicted_pd']   = y_pred_proba
results['predicted_class'] = y_pred_class
results.to_csv('pd_predictions.csv', index=False)
print('Predictions saved: pd_predictions.csv')

print()
print('=== PD Model Summary ===')
print(f'  Model      : Logistic Regression (class_weight=balanced)')
print(f'  Features   : {len(FEATURES)}')
print(f'  Train rows : {X_train.shape[0]:,}')
print(f'  Test rows  : {X_test.shape[0]:,}')
print(f'  AUC-ROC    : {auc:.4f}')
print(f'  KS Stat    : {ks_stat:.4f}')
print()
print('Next step → notebook_03_ecl_engine.ipynb: IFRS 9 ECL Calculator')

---
## 📁 Output Files

| File | Description |
|------|-------------|
| `pd_model_logistic.pkl` | Trained Logistic Regression model |
| `pd_scaler.pkl` | Fitted StandardScaler |
| `pd_predictions.csv` | Test set predictions with PD scores |
| `plot_06_roc_curve.png` | ROC curve with AUC and KS point |
| `plot_07_confusion_matrix.png` | Confusion matrix |
| `plot_08_calibration_curve.png` | Calibration curve |
| `plot_09_feature_importance.png` | Feature coefficients |
| `plot_10_score_distribution.png` | PD score distribution by class |

---
**Next Notebook:** `notebook_03_ecl_engine.ipynb` — IFRS 9 ECL Calculator (ECL = PD × LGD × EAD × Discount Factor)